# Support Vector Machines

Table of contents

✔ Chapter 1.  Support Vector Machines

Chapter 2. Hyperparameter tuning

To install further python libraries, type

`!pip install --target=$my_path [LIBRARY_NAME]`

# Chapter 1-1. Implement from scratch



Download the dataset


*   data source: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data



In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CuV1B9jrXgUwm01zc1aNTDXu35lq3T_1' -O cancer.csv

--2023-10-24 08:09:26--  https://docs.google.com/uc?export=download&id=1CuV1B9jrXgUwm01zc1aNTDXu35lq3T_1
Resolving docs.google.com (docs.google.com)... 209.85.234.100, 209.85.234.138, 209.85.234.101, ...
Connecting to docs.google.com (docs.google.com)|209.85.234.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/do84cemb61h8qes8hsga0v10n89c6f33/1698134925000/12385986347045621890/*/1CuV1B9jrXgUwm01zc1aNTDXu35lq3T_1?e=download&uuid=8792882f-80f2-4e23-aa54-caf5eec37b56 [following]
--2023-10-24 08:09:26--  https://doc-0o-7g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/do84cemb61h8qes8hsga0v10n89c6f33/1698134925000/12385986347045621890/*/1CuV1B9jrXgUwm01zc1aNTDXu35lq3T_1?e=download&uuid=8792882f-80f2-4e23-aa54-caf5eec37b56
Resolving doc-0o-7g-docs.googleusercontent.com (doc-0o-7g-docs.googleusercontent.com)... 173.194.193.132, 2607:f8b0

In [ ]:
import pandas as pd
df = pd.read_csv('cancer.csv') # or use scikit-learn (from sklearn.datasets import load_breast_cancer)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
print(df.shape)

(569, 33)


Preprocess the data

In [ ]:
# transform the categories M and B into 1 and -1, respectively.
diagnosis_map = {'M':1, 'B':-1} # malignant or benign
df['diagnosis'] = df['diagnosis'].map(diagnosis_map)
# drop last column (extra column added by pd) and  first column (id)
df.drop(df.columns[[-1, 0]], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

# create X and Y
Y = df.loc[:, 'diagnosis'].astype(float)
X = df.iloc[:, 1:]
# normalize the features using StandardScaler from sklearn.preprocessing
X_normalized = StandardScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)

Split into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
# insert 1 in every row for intercept b
X.insert(loc=len(X.columns), column='intercept', value=1)

# split data into train and test set
print("splitting dataset into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

splitting dataset into train and test sets...


In [ ]:
from sklearn.model_selection import train_test_split
# insert 1 in every row for intercept b
#X.insert(loc=len(X.columns), column='intercept', value=1)
 # 데이터프레임 X에 'intercept'라는 새로운 열을 생성하고 모든 값 1로 초기화

Define the cost function


*   Note that we are using the hinge loss (refer to Lab2 slides)

In [ ]:
import numpy as np
def cost_function(W, X, Y, C=10000):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = C * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

Define the gradient of the cost function

In [ ]:
def get_cost_gradient(W, X_batch, Y_batch, C=10000):
    # for stochastic gradient descent
    if type(Y_batch) == np.float64:

        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (C * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

Use stochastic gradient descent

In [ ]:
from sklearn.utils import shuffle

def stochastic_gradient_descent(features, outputs, learning_rate=0.000001):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):

        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):

            ascent = get_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = cost_function(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stopping criteria to assume the covergence
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights

            prev_cost = cost
            nth += 1

    return weights

Training

In [ ]:
# train the model
print("training started...")

W = stochastic_gradient_descent(X_train.to_numpy(), y_train.to_numpy())

print("training finished.")
print("weights are: {}".format(W))

training started...
Epoch is: 1 and Cost is: 827.1961717380948
Epoch is: 2 and Cost is: 677.8899299809049
Epoch is: 4 and Cost is: 604.3675270466362
Epoch is: 8 and Cost is: 545.5515845641664
Epoch is: 16 and Cost is: 470.17026050211393
Epoch is: 32 and Cost is: 402.5979009203216
Epoch is: 64 and Cost is: 370.06822369955506
Epoch is: 128 and Cost is: 345.5522392635492
Epoch is: 256 and Cost is: 315.35145379922244
Epoch is: 512 and Cost is: 270.5815820746626
Epoch is: 1024 and Cost is: 258.3053801327621
Epoch is: 2048 and Cost is: 244.18778863414548
Epoch is: 4096 and Cost is: 244.90606830342566
training finished.
weights are: [ 0.19285473 -0.02725298 -0.52146112  0.09644568 -0.25433655 -3.68413206
  2.34575158  3.92158221 -0.27394286  1.64911728  3.57592977 -0.85897516
 -1.80567604  1.75100534  0.77129926  1.19908123 -2.39361293  1.56757516
 -1.03651227 -2.69162755  2.14059745  2.21792485 -1.08526256  2.10269087
 -0.34303793 -0.5962708   2.55573029 -0.19268533  1.87262151  0.5061519
  

In [ ]:
# testing the model
from sklearn.metrics import accuracy_score, recall_score, precision_score
print("testing the model...")
y_train_predicted = np.array([])
for i in range(X_train.shape[0]):
    yp = np.sign(np.dot(X_train.to_numpy()[i], W))
    y_train_predicted = np.append(y_train_predicted, yp)

y_test_predicted = np.array([])
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(X_test.to_numpy()[i], W))
    y_test_predicted = np.append(y_test_predicted, yp)

print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
print("precision on test dataset: {}".format(precision_score(y_test, y_test_predicted)))

testing the model...
accuracy on test dataset: 0.9649122807017544
recall on test dataset: 0.9767441860465116
precision on test dataset: 0.9333333333333333


# Chapter 1-2. Use scikit-learn library


Use linear kernel

In [ ]:
from sklearn.svm import LinearSVC, SVC

X_train.columns = X_train.columns.astype(str)

classifier = LinearSVC(C=10000, loss='hinge')
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=10000, loss='hinge')

In [ ]:
X_test.columns = X_test.columns.astype(str)

y_pred = classifier.predict(X_test)

In [ ]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("precision on test dataset: {}".format(precision_score(y_test, y_pred)))

accuracy on test dataset: 0.9385964912280702
recall on test dataset: 0.9767441860465116
precision on test dataset: 0.875


Use rbf kernel

In [ ]:
classifier_w_rbf = SVC(kernel = 'rbf', random_state = 0, C=10000)
classifier_w_rbf.fit(X_train, y_train)

SVC(C=10000, random_state=0)

In [ ]:
y_pred = classifier_w_rbf.predict(X_test)


In [ ]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, y_pred)))

accuracy on test dataset: 0.9385964912280702
recall on test dataset: 0.9534883720930233
precision on test dataset: 0.9534883720930233


Use cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(classifier_w_rbf, testX, y, cv=5)

In [ ]:
print('cross-val-score: {}'.format(scores))
print('cross-val-score.mean: {}'.format(scores.mean()))

cross-val-score: [0.95652174 0.91304348 1.         0.91304348 0.90909091]
cross-val-score.mean: 0.9383399209486166
